In [48]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [49]:
path = '/content/drive/MyDrive/DATA/데이터분석캡스톤디자인/'

In [50]:
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [51]:
data = pd.read_csv(os.path.join(path+'recipe_메인반찬2.csv'),encoding='utf-8')
data_rep = pd.read_csv(os.path.join(path+'대표메뉴_메인반찬.csv'),encoding='utf-8')

new_data = data[['레시피 제목','요리재료']]

new_data.rename(columns = {'레시피 제목' : '대표메뉴명'}, inplace = True)
new_data = pd.concat([data_rep, new_data])

tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(new_data['요리재료'])

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

indices = pd.Series(data_rep.index, index = data_rep['대표메뉴명']).drop_duplicates()

def get_recommendations(recipe, cosine_sim = cosine_sim):
    # 선택한 레시피로부터 해당되는 인덱스를 받아온다.
    recipe2idx = {}
    for i, c in enumerate(new_data["대표메뉴명"]) :
        recipe2idx[i] = c
    idx2recipe = {}
    for i, c in recipe2idx.items() :
        idx2recipe[c] = i
    
    # 모든 레시피에 대해서 해당 레시피와의 유사도 계산
    idx = idx2recipe[recipe]
    sim_scores = [(i, c) for i, c in enumerate(cosine_sim[idx]) if i != idx]

    # 유사도에 따라 레시피들을 정렬
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse=True)

    new_sim_scores = [i for i in sim_scores if i[1] > 0.25]

    # 가장 유사한 10개의 레시피의 인덱스
    recipe_indices = [i[0] for i in new_sim_scores]

    return new_data[["대표메뉴명", "요리재료"]].iloc[recipe_indices]

user_idx0_data = get_recommendations('제육볶음').head(10) # 대표메뉴 0 번을 선택한 사용자의 추천 리스트
user_idx1_data = get_recommendations('오징어볶음').head(10) # 대표메뉴 1 번을 선택한 사용자의 추천 리스트
user_idx2_data = get_recommendations('소불고기').head(10) # 대표메뉴 2 번을 선택한 사용자의 추천 리스트
user_idx3_data = get_recommendations('갈비찜').head(10) # 대표메뉴 3 번을 선택한 사용자의 추천 리스트

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [52]:
user_idx0_data

,대표메뉴명,요리재료
20320,돼지고기 잡채,돼지고기 양파 당근 대파 표고버섯 파프리카 청양고추 참기름 게간장 간마늘 설탕 후추...
18660,맵지 않은 두부 간장 조림,두부 소금 카놀라유 간장 물 참치액 설탕 올리고당 후추 다진마늘 참기름 볶음깨
4932,고추장 불고기,돼지고기 대파 당근 양파 설탕 맛술 간장 다진마늘 고추장
13097,자취생 간단 제육볶음!!,다진마늘 간장 고춧가루 설탕 맛술 고추장 참기름 돼지고기 양파 대파
10637,기사식당 스타일 돼지고기 간장 불고기,돼지고기 양파 대파 맛술 설탕 다진마늘 참기름 후추
13910,두반장없이 쉽게 만드는 마파두부!,두부 돼지고기 양파 대파 고추장 된장 고춧가루 다진마늘 설탕 간장
15050,돼지불백은 나눌수록 더 맛있잖아요? 화제의 무한도전 돼지불백 황금레시피,돼지고기 대파 깨 간장 설탕 맛술 참기름 다진마늘 후추
19844,닭볶음탕,닭고기 양파 당근 감자 애호박 파 표고버섯 게간장 미림 다진마늘 생강 고추장 고추가...
7899,간장 돼지불고기 만드는 법 레시피,돼지고기 양파 대파 당근 당면 진간장 맛술 꿀 다진마늘 참기름 소금 후추 맛술
18347,<세종댁달순 말복 추천 레시피> 새콤달콤한 소불고기 만들기,소고기 양파 당근 새송이 붉은 풋고추 참기름 후추 간장 참치액 설탕 다짐 요리당 참...


In [53]:
# 각 사용자가 추천받은 메뉴의 리스트 만들기

def make_user_list(data):
    user_list = []
    for i in range(len(data.iloc[:,0])):
      user_list.append(data.iloc[i,0])
    return user_list

user0_list = make_user_list(user_idx0_data)
user1_list = make_user_list(user_idx1_data)
user2_list = make_user_list(user_idx2_data)

In [54]:
# 사용자들의 모든 메뉴 합치기

def make_list(user_list,total_list):
  for i in range(len(user_list)):
    total_list.append(user_list[i])
  return total_list

total_list = []
make_list(user0_list,total_list)
make_list(user1_list,total_list)
make_list(user2_list,total_list)

total_set = set(total_list) # 집합으로 변환 (중복 제거)
total_list = list(total_set) # 리스트로 변환

print(total_list)
print(len(total_list))

['닭볶음탕', 'TV요리) 골뱅이무침 쫄깃쫄깃 술이들어간다~ 쭉-쭉쭉쭉! ', '쇠고기 표고버섯볶음 만드는 법~표고버섯 듬뿍 넣어서**', '오징어미나리무침 만드는법  오징어미나리초무침 ', '고추장 불고기 ', '두반장없이 쉽게 만드는 마파두부!', '정말 편하고 간단하게 만드는 삼겹살 보쌈', '우리 집 메인 반찬 새송이 쇠고기 볶음', '소불고기 만들기', '밥도둑 매콤 오징어볶음', '자취생 간단 제육볶음!!', '쇠고기산적 만드는 법', '와인안주 _ 오징어볶음 소면도 잘 어울리네', '백종원 오징어볶음 맛있게 만들어먹어요~!!', '간장양념이 맛나는!너비아니구이', '골뱅이무침 만들기', '특별한 맛  유자청 한우 불고기', "[셜록] 백종원 '불고기' 이것만 알면 요리 잘하는 사람", '맵지 않은 두부 간장 조림', '오징어볶음', '소불고기 만드는 법 표고버섯 소고기 불고기 든든한 집밥', '기사식당 스타일 돼지고기 간장 불고기', '간장 돼지불고기 만드는 법 레시피', '돼지불백은 나눌수록 더 맛있잖아요? 화제의 무한도전 돼지불백 황금레시피', '돼지고기 잡채', '오징어볶음 간단하게 후다닥~!', '<세종댁달순 말복 추천 레시피> 새콤달콤한 소불고기 만들기', '쇠불고기', '매콤한 오징어볶음 만들기', '오징어볶음 만들기. 냉장고파먹기. 백종원 오징어 볶음 따라하다 말았다..']
30


In [55]:
# 각 사용자의 메뉴 리스트와 total 메뉴 리스트 비교하기

def compare_user_total(user_list,total_list):
  user_bin = []
  total_list = sorted(total_list)
  user_list = sorted(user_list)

  for j in range(len(total_list)):
    if total_list[j] in user_list :
      user_bin.append(1)
    elif total_list[j] not in user_list :
      user_bin.append(0)
  
  return user_bin

user0_bin = compare_user_total(user0_list,total_list)
user1_bin = compare_user_total(user1_list,total_list)
user2_bin = compare_user_total(user2_list,total_list)

print(user0_bin)
print(user1_bin)
print(user2_bin)

[1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0]
[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1]


In [56]:
# 행:user,열:menu인 행렬 만들기 (menu가 추천되었으면 1, 아니면 0)

data = [user0_bin,user1_bin,user2_bin]
compare_df = pd.DataFrame(data,columns=total_list)

compare_df

,닭볶음탕,TV요리) 골뱅이무침 쫄깃쫄깃 술이들어간다~ 쭉-쭉쭉쭉!,쇠고기 표고버섯볶음 만드는 법~표고버섯 듬뿍 넣어서**,오징어미나리무침 만드는법 오징어미나리초무침,고추장 불고기,두반장없이 쉽게 만드는 마파두부!,정말 편하고 간단하게 만드는 삼겹살 보쌈,우리 집 메인 반찬 새송이 쇠고기 볶음,소불고기 만들기,밥도둑 매콤 오징어볶음,...,소불고기 만드는 법 표고버섯 소고기 불고기 든든한 집밥,기사식당 스타일 돼지고기 간장 불고기,간장 돼지불고기 만드는 법 레시피,돼지불백은 나눌수록 더 맛있잖아요? 화제의 무한도전 돼지불백 황금레시피,돼지고기 잡채,오징어볶음 간단하게 후다닥~!,<세종댁달순 말복 추천 레시피> 새콤달콤한 소불고기 만들기,쇠불고기,매콤한 오징어볶음 만들기,오징어볶음 만들기. 냉장고파먹기. 백종원 오징어 볶음 따라하다 말았다..
0,1,0,0,1,0,1,0,1,1,1,...,0,0,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,1,0,0,0,...,0,1,1,1,1,1,0,0,0,0
2,0,0,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1


In [57]:
# 행 벡터들 간의 유사도 구하기 (user들 간의 유사도 구하기)

cos_sim_df = pd.DataFrame(cosine_similarity(compare_df,compare_df)) # 지금 예시에서는 사용자가 추천받은 메뉴 리스트가 모두 달라서 이렇게 나옴
cos_sim_df

,0,1,2
0,1.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,1.0


In [60]:
# 유사도 행렬의 대각 행렬 기준으로 평균값 구하기
# 합 = (전체 합 - user수) / 2

def mean_sim(cos_sim_df):

  total = 0
  for i in range(len(cos_sim_df)):
    col = cos_sim_df.iloc[i]
    total += sum(col)
  i = 1
  bottom = 0
  while i < len(cos_sim_df):
    bottom += i
    i += 1

  mean_sim_total = (total-len(cos_sim_df))/2
  mean_sim = mean_sim_total / bottom # 평균값

  return mean_sim

print(mean_sim(cos_sim_df))  # 지금 적용한 것에서는 합이 0이라서 결과가 이렇게 나옴

-7.401486830834377e-17


In [59]:
# personalization 값

personalization = 1 - mean_sim
personalization

1.0